In [ ]:
import ee
import geemap
import pandas as pd
from shapely.wkt import loads  

# initialize Earth Engine
ee.Initialize()

# load CSV using Pandas
df = pd.read_csv("../notebook_outline_version/data/Costa_Classification_Data_Cleaned2.csv")

# ensure the dataset has a 'geometry' column with WKT format
if "geometry" not in df.columns:
    raise ValueError("The dataset must have a 'geometry' column with WKT format.")

# handle NaN values (replace with empty strings or drop them)
df = df.fillna("")  # Use dropna() if you want to remove rows with NaNs

# Convert dataframe to a list of features
features = []
for _, row in df.iterrows():
    geometry = loads(row["geometry"])  
    if geometry.geom_type == "Point":  
        lon, lat = geometry.x, geometry.y  
        properties = {k: v for k, v in row.to_dict().items() if pd.notna(v)}  # Remove NaN values
        feature = ee.Feature(ee.Geometry.Point([lon, lat]), properties)  
        features.append(feature)


# batch processing with smaller batches (e.g., 200)
batch_size = 200  
feature_batches = [features[i:i+batch_size] for i in range(0, len(features), batch_size)]

# create multiple FeatureCollections
datasets = [ee.FeatureCollection(batch) for batch in feature_batches]

# define color mapping for vegetation types
vegetation_colors = ee.Dictionary({
    "Tree": "#1b5e20",
    "Not_Applicable": "#76ff03",
    "Wetland": "#004d40",
    "Palms": "#8d6e63",
    "Hebacias": "#ffeb3b",
    "Herbaceous": "#fcba03",
    "Plastico": "#fc62d3"
})

# function to style features
def style_feature(feature):
    color = vegetation_colors.get(feature.get("Vegetation"), "#ffffff")  
    return feature.set({"style": ee.Dictionary({"color": color})})

# create a map
Map = geemap.Map()

# add each batch separately to avoid large requests
for i, dataset in enumerate(datasets):
    styled_dataset = dataset.map(style_feature).style(**{"styleProperty": "style"})
    Map.addLayer(styled_dataset, {}, f"Vegetation Classification Batch {i+1}")

# display map
Map
